In [1]:
# Import the Libraries for Google Drive Document Retrieval
from pydrive.auth import GoogleAuth
import pandas as pd
from pydrive.drive import GoogleDrive
import glob

#Libraries to handel and prepare data 
import importlib
import datetime
import holidays
import progressbar
from time import sleep
import pycountry
import math

In [2]:
# A browser window will open. login using the appropriate account.
gauth = GoogleAuth()
gauth.LocalWebserverAuth() #

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=267819126046-3da5rmgml4iemor49ermcv23tnaj02t5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [3]:
drive = GoogleDrive(gauth)

In [4]:
drive

## Holidays auxiliary code

In [5]:
#Evaluates wether a given date fro a given country is a national holiday or not.
def getHoliday(country, date):
    
    res=False
    if hasattr(holidays, country):
        function_string = 'holidays.' + country
        mod_name, func_name = function_string.rsplit('.',1)
        mod = importlib.import_module(mod_name)
        func = getattr(mod, func_name)
        if(isinstance(func, int)==False):
            result = func()
            res = result.get(date) is not None
 
    return res

## Retrieve data from Google Drive

### Read single file 

In [6]:
#Reads a csv file in a certain drive folder, skipping n first nrows. 
#Returns a dataframe, if not found returns empty dataframe
def read_csv_file(folder, fileName, nrows):
    
    listed_mortality = drive.ListFile({'q': "title contains '" + fileName + ".csv' and '" + folder + "' in parents"}).GetList()
    
    if len(listed_mortality)>0:
        title = listed_mortality[0]['title']
        id = listed_mortality[0]['id']
        each_file = drive.CreateFile({'id': id})
        each_file.GetContentFile(title)
        df = pd.read_csv(title, skiprows=nrows)
    else:
        
        df=pd.DataFrame()
    
    return df

### Grouping policies 

Method used to group policies as the average of policies in the selected week. It has been decided to group values on Sundays, since excess mortality data comes out every Sunday.
The rest of the variables are calculated as detailed below:
* Day x Policies = policies and trends of x - 18 days.
* Week x Policies = mean of policies and trends in the selected week.

Every day in the week has a policies related from 18 previous days (average death limit for covid death in patients)

In [7]:
def group_policies(filename):

    df = pd.read_csv(filename)
    df['Holiday']=df.apply(lambda x: getHoliday(x['Code'], x['Day']), axis=1)
    df['Date'] = pd.to_datetime(df['Day']) + pd.to_timedelta(18, unit='d')
    
    #Group to closest Monday
    df = df.groupby(['Code', pd.Grouper(key='Date', freq='W-SUN')]).mean().reset_index()
    #df = df.groupby(['Code', pd.Grouper(key='Date', freq='W-MON', label='left',closed='left')]).mean().reset_index()
        
    return df

Loop through all files contained in folder and aggregate them using "group_policies" auxiliary method as detailed above.

In [9]:
def df_group_policies_trends_others():
    listed = drive.ListFile({'q': "title contains '.csv' and '1XpkEI-8Zfe0IQCzoa9Pas6Y_hpjGVhGE' in parents"}).GetList()
    list_id = []
    list_title = []
    for file in listed:
        list_id.append(file['id'])
        list_title.append(file['title'])
        
    bar = progressbar.ProgressBar(maxval=15, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    
    df = pd.DataFrame()

    i=0

    for id, title in zip(list_id, list_title):
        print(title)
        bar.update(i+1)
        sleep(0.1)
        
        each_file = drive.CreateFile({'id': id})
        each_file.GetContentFile(title)

        if(i==0):
            df = group_policies(title)
        else:
            df_aux = group_policies(title)
            df = pd.merge(df, df_aux, on=["Code", "Date"])
        i=i+1
            
    return df

# Create Dataframe with all Features

## Policies, Mobility Trends and Holidays

The call to this method reads and groups data as described in df_group_policies_trends_others and may take some time

In [10]:
%%time
df_policies=df_group_policies_trends_others()

covid-contact-tracing.csv


changes-visitors-covid.csv


debt-relief-covid.csv


income-support-covid.csv


covid-19-testing-policy.csv


...

stay-at-home-covid.csv


public-gathering-rules-covid.csv


public-events-covid.csv


workplace-closures-covid.csv


school-closures-covid .csv
CPU times: user 6min 37s, sys: 3.06 s, total: 6min 40s
Wall time: 7min 42s


In [11]:
#Delete duplicated Holiday columns and view output
del df_policies['Holiday_x']
del df_policies['Holiday_y']
df_policies['Holiday']=df_policies['Holiday']*7
df_policies

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,restrictions_internal_movements,close_public_transport,public_information_campaigns,facial_coverings,stay_home_requirements,restriction_gatherings,cancel_public_events,workplace_closures,school_closures,Holiday
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.0
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,0.0,0.000000,0.285714,0.0,0.0,0.0,0.0,0.0,1.285714,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,0.0,1.000000,2.000000,3.0,2.0,3.0,2.0,2.0,1.000000,0.0
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,0.0,1.000000,2.000000,3.0,2.0,3.0,2.0,2.0,1.000000,0.0
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,0.0,1.000000,2.000000,3.0,2.0,3.0,2.0,2.0,1.000000,0.0
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,0.0,0.571429,2.000000,3.0,2.0,3.0,2.0,2.0,1.000000,0.0


In [12]:
#Lets check for Holiday in Spain related to previous 18 days.
df_policies[(df_policies['Code']=='ESP') & (df_policies['Holiday'] > 0)]

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,restrictions_internal_movements,close_public_transport,public_information_campaigns,facial_coverings,stay_home_requirements,restriction_gatherings,cancel_public_events,workplace_closures,school_closures,Holiday
2189,ESP,2020-05-24,1.0,-84.897857,-46.714286,26.448857,-73.347000,-58.612286,-63.204000,2.0,...,1.0,1.0,2.0,0.857143,2.0,4.0,2.0,3.0,3.0,1.0
2204,ESP,2020-09-06,1.0,-16.795857,-4.857000,3.489571,-35.918429,66.857143,-31.877571,2.0,...,2.0,0.0,2.0,3.000000,1.0,4.0,2.0,2.0,2.0,1.0
2212,ESP,2020-11-01,1.0,-24.816286,-3.571571,5.204000,-24.102143,6.000000,-20.979714,2.0,...,2.0,0.0,2.0,3.285714,1.0,4.0,2.0,2.0,3.0,1.0
2215,ESP,2020-11-22,1.0,-33.673571,-1.632714,8.020286,-27.571429,-7.714286,-19.734714,2.0,...,2.0,0.0,2.0,4.000000,2.0,4.0,2.0,2.0,1.0,1.0
2220,ESP,2020-12-27,1.0,-32.979429,-0.265286,9.469429,-29.898000,-19.550857,-22.979714,2.0,...,2.0,0.0,2.0,4.000000,2.0,4.0,2.0,2.0,1.0,2.0
2223,ESP,2021-01-17,1.0,-28.591857,2.510143,10.469429,-34.102143,-12.571429,-37.040857,2.0,...,2.0,0.0,2.0,4.000000,2.0,4.0,2.0,2.0,3.0,1.0
2224,ESP,2021-01-24,1.0,-31.203857,-1.571571,12.693857,-40.061143,-16.612143,-43.591857,2.0,...,2.0,0.0,2.0,4.000000,2.0,4.0,2.0,2.0,3.0,2.0


## Weather Data

Collects weather data from all stations in the world and relate them to each country taking average of values. The treatment and creation of this file can be found in the file weather_data which uses Google BigQuery and Google Maps API in order to retrieve all the desired information

In [13]:
df_weather=read_csv_file('1yXs7YOhttKS7STixDmqD1kv1woRYWfUd','weather_final_df',0)
df_weather=df_weather.dropna()
#Cast country code to standard used in our dataset
for country in pycountry.countries: 
    df_weather.loc[df_weather['country']==country.alpha_2, 'Code'] = country.alpha_3
#Apply grouping
df_weather=df_weather[['Code','date', 'temp', 'prcp']]
df_weather['Date'] = pd.to_datetime(df_weather['date']) + pd.to_timedelta(18, unit='d')
df_weather = df_weather.groupby(['Code', pd.Grouper(key='Date', freq='W-SUN')]).mean().reset_index()
df_weather

In [17]:
df_merge=df_policies

In [18]:
#Combine original dataframe with retrieved information
df_merge = pd.merge(df_merge, df_weather, on=["Code", "Date"], how="left")
df_merge.head()

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,public_information_campaigns,facial_coverings,stay_home_requirements,restriction_gatherings,cancel_public_events,workplace_closures,school_closures,Holiday,temp,prcp
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,NaN
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,NaN,NaN
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,NaN
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,NaN
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,0.285714,0.0,0.0,0.0,0.0,0.0,1.285714,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,2.000000,3.0,2.0,3.0,2.0,2.0,1.000000,0.0,23.986111,0.0
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,2.000000,3.0,2.0,3.0,2.0,2.0,1.000000,0.0,NaN,NaN
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,2.000000,3.0,2.0,3.0,2.0,2.0,1.000000,0.0,NaN,NaN
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,2.000000,3.0,2.0,3.0,2.0,2.0,1.000000,0.0,NaN,NaN


## Doctors and Nurses

Doctors and nurses information per 1000 

In [19]:
doctors_df=read_csv_file('17wrWTb6BZMqWkNrqYipTQXo7IOxqekOf','doctors',0)
nurses_df=read_csv_file('17wrWTb6BZMqWkNrqYipTQXo7IOxqekOf','nurses',0)

doctors_df=doctors_df[['LOCATION', 'TIME', 'Value']]
doctors_df_18=doctors_df[doctors_df['TIME'] == 2018]
nurses_df=nurses_df[['LOCATION', 'TIME', 'Value']]
nurses_df=nurses_df[nurses_df['TIME'] == 2018]

for index, row in doctors_df.iterrows(): 
    df_merge.loc[df_merge['Code']==row.LOCATION , 'doctors_per_1000'] = row.Value
for index, row in nurses_df.iterrows(): 
    df_merge.loc[df_merge['Code']==row.LOCATION , 'nurses_per_1000'] = row.Value
df_merge

## Hospital Beds

Hospital beds per 1000 

In [28]:
hospital_beds_df=read_csv_file('1YlfutCzBqYwPhZw-7OsfLC3k1uv1LYfa','hospital_beds', 3)
hospital_beds_df[hospital_beds_df['Country Code']=='ESP']
#Select latest value between 2010 and 2020 
years=hospital_beds_df.columns[54:-1]

latest_values=[]

#Get latest value for hospital beds
for index, row in hospital_beds_df.iterrows():
    
    aux_year=2010
    
    for year in years:
        
        if((math.isnan(row[year])==False)):
            
            aux_year=year
            
    latest_values.append(row[str(aux_year)])

hospital_beds_df['latest_value']=latest_values
hospital_beds_df=hospital_beds_df[['Country Code', 'latest_value']]
hospital_beds_df.columns=['Code', 'beds_per_1000']
hospital_beds_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
68,Spain,ESP,"Hospital beds (per 1,000 people)",SH.MED.BEDS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,2.99,2.96,2.97,2.98,2.97,2.97,2.97,NaN,NaN,NaN


In [32]:
df_merge = pd.merge(df_merge, hospital_beds_df, on=["Code"], how="left")
df_merge.head()

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,restriction_gatherings,cancel_public_events,workplace_closures,school_closures,Holiday,temp,prcp,doctors_per_1000,nurses_per_1000,beds_per_1000
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,0.0,0.0,0.0,0.000000,1.0,NaN,NaN,NaN,NaN,NaN
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,0.0,0.0,0.0,1.285714,1.0,NaN,NaN,NaN,NaN,NaN


## International Tourism

Collect international toruism data from previous years and select last occurence 

In [33]:
tourism_df=read_csv_file('1L_YnKR2utwlvzE2ykY47xGsZeLwi3RKu','international-tourism-number-of-arrivals', 0)

tourism_df=tourism_df.dropna()
years=tourism_df['Year'].unique()
countries=tourism_df['Code'].unique()

#Iterate through all values and collect the values for the maximum year recorded for the selected country
latest_values=[]

max_year=1995

for country in countries:
    max_year=1995
    for year in years:
        if((year > max_year) & (len(tourism_df[(tourism_df['Year']==year) & (tourism_df['Code']==country)]) > 0)):
            max_year = year

    latest_values.append(tourism_df[(tourism_df['Year']==max_year) & (tourism_df['Code']==country)]['International tourism, number of arrivals'].values[0])

tourism_df_latest=pd.DataFrame()
tourism_df_latest['Code']=countries
tourism_df_latest['number_of_arrivals']=latest_values
tourism_df_latest  

,Entity,Code,Year,"International tourism, number of arrivals"
0,Albania,ALB,1995,304000.0
1,Albania,ALB,1996,287000.0
2,Albania,ALB,1997,119000.0
3,Albania,ALB,1998,184000.0
4,Albania,ALB,1999,371000.0


In [39]:
df_merge = pd.merge(df_merge, tourism_df_latest, on=["Code"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,cancel_public_events,workplace_closures,school_closures,Holiday,temp,prcp,doctors_per_1000,nurses_per_1000,beds_per_1000,number_of_arrivals
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,0.0,0.0,0.000000,1.0,NaN,NaN,NaN,NaN,NaN,1102000.0
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,0.0,0.0,1.285714,1.0,NaN,NaN,NaN,NaN,NaN,1102000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,2.0,2.0,1.000000,0.0,23.986111,0.0,NaN,NaN,1.7,2168000.0
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,2.0,2.0,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,2.0,2.0,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,2.0,2.0,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0


## Urbanization

Collect % urbanization population data in each country from previous years and select last occurence  

In [40]:
urbanization_df=read_csv_file('1XrFzNCmJRUYxATxI6gSfwXrk9aXjYP2K','share-of-population-urban', 0)

urbanization_df=urbanization_df.dropna()
years=urbanization_df['Year'].unique()
countries=urbanization_df['Code'].unique()

#Iterate through all values and collect the values for the maximum year recorded for the selected country
latest_values=[]

for country in countries:
    max_year=2010
    for year in years:
        if((year > max_year) & (len(urbanization_df[(urbanization_df['Year']==year) & (urbanization_df['Code']==country)]) > 0)):
            max_year = year

    latest_values.append(urbanization_df[(urbanization_df['Year']==max_year) & (urbanization_df['Code']==country)]['Urban population (% of total)'].values[0])

urbanization_df_latest=pd.DataFrame()
urbanization_df_latest['Code']=countries
urbanization_df_latest['urban_population']=latest_values
urbanization_df_latest

,Entity,Code,Year,Urban population (% of total)
0,Afghanistan,AFG,1960,8.401
1,Afghanistan,AFG,1961,8.684
2,Afghanistan,AFG,1962,8.976
3,Afghanistan,AFG,1963,9.276
4,Afghanistan,AFG,1964,9.586
...,...,...,...,...
15067,Zimbabwe,ZWE,2013,32.654
15068,Zimbabwe,ZWE,2014,32.504
15069,Zimbabwe,ZWE,2015,32.385
15070,Zimbabwe,ZWE,2016,32.296


In [46]:
df_merge = pd.merge(df_merge, urbanization_df_latest, on=["Code"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,workplace_closures,school_closures,Holiday,temp,prcp,doctors_per_1000,nurses_per_1000,beds_per_1000,number_of_arrivals,urban_population
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,0.0,0.000000,1.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,0.0,1.285714,1.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,2.0,1.000000,0.0,23.986111,0.0,NaN,NaN,1.7,2168000.0,32.237
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,2.0,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,2.0,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,2.0,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237


## Vaccination

Retrieve vaccination data grouping done by week 

In [47]:
vaccination_df=read_csv_file('1mLYB7CX0BYOkGqRp6QWhDKV_FrXOlDIr','covid-vaccination-doses-per-capita', 0)
vaccination_df['Date'] = pd.to_datetime(vaccination_df['Day']) + pd.to_timedelta(18, unit='d')
vaccination_df=vaccination_df[['Code', 'Date', 'total_vaccinations_per_hundred']]
vaccination_df = vaccination_df.groupby(['Code', pd.Grouper(key='Date', freq='W-SUN')]).mean().reset_index()
vaccination_df.columns=['Code', 'Date', 'total_vaccinations_per_100']
vaccination_df

,Code,Date,total_vaccinations_per_100
0,AFG,2021-03-14,0.000000
1,AFG,2021-03-21,0.020000
2,AFG,2021-04-04,0.140000
3,AGO,2021-03-21,0.000000
4,AGO,2021-03-28,0.020000
...,...,...,...
1313,ZWE,2021-03-28,0.225714
1314,ZWE,2021-04-04,0.250000
1315,ZWE,2021-04-11,0.294286
1316,ZWE,2021-04-18,0.518571


In [48]:
df_merge = pd.merge(df_merge, vaccination_df, on=["Code", "Date"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,school_closures,Holiday,temp,prcp,doctors_per_1000,nurses_per_1000,beds_per_1000,number_of_arrivals,urban_population,total_vaccinations_per_100
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,0.000000,1.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,1.285714,1.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,1.000000,0.0,23.986111,0.0,NaN,NaN,1.7,2168000.0,32.237,0.518571
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,0.813333
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,NaN
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,NaN


## Youth Unemployment

#### Youth unemployment data for all countries, selecting last value occurence

In [49]:
youth_unemployment_df=read_csv_file('1Vv6uPieFDONx3kSA9V6vbgzJSireUJDN','youth_unemployment', 3)

#Select latest value between 2010 and 2020 
years=youth_unemployment_df.columns[54:-1]

#Iterate through all values and collect the values for the maximum year recorded for the selected country
latest_values=[]

for index, row in youth_unemployment_df.iterrows():
    
    aux_year=2010
    
    for year in years:
        
        if((math.isnan(row[year])==False)):
            
            aux_year=year
            
    latest_values.append(row[str(aux_year)])
    
youth_unemployment_df['latest_value']=latest_values
youth_unemployment_df=youth_unemployment_df[['Country Code', 'latest_value']]
youth_unemployment_df.columns=['Code', '%youth_unemployment_total']
youth_unemployment_df

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,"Unemployment, youth total (% of total labor fo...",SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,"Unemployment, youth total (% of total labor fo...",SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,18.090000,18.070000,17.889999,17.780001,17.680000,17.500000,17.330000,17.219999,NaN,NaN
2,Angola,AGO,"Unemployment, youth total (% of total labor fo...",SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,16.940001,17.000000,16.879999,16.809999,16.690001,16.510000,16.360001,16.260000,NaN,NaN
3,Albania,ALB,"Unemployment, youth total (% of total labor fo...",SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,28.620001,30.959999,39.529999,39.599998,35.630001,30.910000,28.209999,26.990000,NaN,NaN
4,Andorra,AND,"Unemployment, youth total (% of total labor fo...",SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,"Unemployment, youth total (% of total labor fo...",SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,"Yemen, Rep.",YEM,"Unemployment, youth total (% of total labor fo...",SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,23.270000,23.990000,24.469999,24.309999,24.260000,24.250000,24.230000,24.240000,NaN,NaN
261,South Africa,ZAF,"Unemployment, youth total (% of total labor fo...",SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,51.389999,51.310001,51.340000,50.310001,53.630001,53.529999,53.770000,57.470001,NaN,NaN
262,Zambia,ZMB,"Unemployment, youth total (% of total labor fo...",SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,16.440001,17.610001,18.639999,19.690001,20.709999,21.620001,22.700001,22.629999,NaN,NaN


In [53]:
df_merge = pd.merge(df_merge, youth_unemployment_df, on=["Code"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,Holiday,temp,prcp,doctors_per_1000,nurses_per_1000,beds_per_1000,number_of_arrivals,urban_population,total_vaccinations_per_100,%youth_unemployment_total
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,1.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,1.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,0.0,23.986111,0.0,NaN,NaN,1.7,2168000.0,32.237,0.518571,8.13
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,0.813333,8.13
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,NaN,8.13
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,NaN,8.13


## Life Expectancy

In [54]:
life_expectancy_df=read_csv_file('1946XP73K-KBKOYjZMTrR-kObEkr7VZ23','life_expectancy', 3)

#Select latest value between 2010 and 2020 
years=life_expectancy_df.columns[54:-1]

#Iterate through all values and collect the values for the maximum year recorded for the selected country
latest_values=[]

for index, row in life_expectancy_df.iterrows():
    
    aux_year=2010
    
    for year in years:
        
        if((math.isnan(row[year])==False)):
            
            aux_year=year
            
    latest_values.append(row[str(aux_year)])
    
life_expectancy_df['latest_value']=latest_values
life_expectancy_df=life_expectancy_df[['Country Code', 'latest_value']]
life_expectancy_df.columns=['Code', 'life_expectancy']
life_expectancy_df

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,65.662,66.074,66.444,66.787,67.113,67.435,...,75.299000,75.441000,75.583000,75.725000,75.868000,76.010000,76.152000,NaN,NaN,NaN
1,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,32.446,32.962,33.471,33.971,34.463,34.948,...,62.054000,62.525000,62.966000,63.377000,63.763000,64.130000,64.486000,NaN,NaN,NaN
2,Angola,AGO,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,37.524,37.811,38.113,38.430,38.760,39.102,...,57.236000,58.054000,58.776000,59.398000,59.925000,60.379000,60.782000,NaN,NaN,NaN
3,Albania,ALB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,62.283,63.301,64.190,64.914,65.463,65.850,...,77.252000,77.554000,77.813000,78.025000,78.194000,78.333000,78.458000,NaN,NaN,NaN
4,Andorra,AND,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,NaN,NaN,NaN,NaN,NaN,NaN,...,70.497561,70.797561,71.097561,71.346341,71.646341,71.946341,72.195122,NaN,NaN,NaN
260,"Yemen, Rep.",YEM,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,29.919,30.163,30.500,30.943,31.501,32.175,...,65.920000,66.016000,66.066000,66.085000,66.087000,66.086000,66.096000,NaN,NaN,NaN
261,South Africa,ZAF,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,48.406,48.777,49.142,49.509,49.888,50.284,...,60.060000,61.099000,61.968000,62.649000,63.153000,63.538000,63.857000,NaN,NaN,NaN
262,Zambia,ZMB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,46.687,47.084,47.446,47.772,48.068,48.351,...,58.502000,59.746000,60.831000,61.737000,62.464000,63.043000,63.510000,NaN,NaN,NaN


In [58]:
df_merge = pd.merge(df_merge, life_expectancy_df, on=["Code"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,temp,prcp,doctors_per_1000,nurses_per_1000,beds_per_1000,number_of_arrivals,urban_population,total_vaccinations_per_100,%youth_unemployment_total,life_expectancy
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,23.986111,0.0,NaN,NaN,1.7,2168000.0,32.237,0.518571,8.13,61.195
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,0.813333,8.13,61.195
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,NaN,8.13,61.195
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,NaN,8.13,61.195


## %Of Population greater than 65

In [59]:
df_population_gr_65=read_csv_file('1z4gqhcx55nLbhJ9v6__Re4UEglUt7-Dz','ages65_and_older', 3)
years=df_population_gr_65.columns[54:-1]

latest_values=[]

for index, row in df_population_gr_65.iterrows():
    
    aux_year=2010
    
    for year in years:
        
        if((math.isnan(row[year])==False)):
            
            aux_year=year
            
    latest_values.append(row[str(aux_year)])
    
df_population_gr_65['latest_value']=latest_values
df_population_gr_65=df_population_gr_65[['Country Code', 'latest_value']]
df_population_gr_65.columns=['Code', '%df_population_gr_65']
df_population_gr_65
df_merge = pd.merge(df_merge, df_population_gr_65, on=["Code"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,prcp,doctors_per_1000,nurses_per_1000,beds_per_1000,number_of_arrivals,urban_population,total_vaccinations_per_100,%youth_unemployment_total,life_expectancy,%df_population_gr_65
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,0.0,NaN,NaN,1.7,2168000.0,32.237,0.518571,8.13,61.195,2.980608
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,NaN,NaN,NaN,1.7,2168000.0,32.237,0.813333,8.13,61.195,2.980608
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,NaN,NaN,NaN,1.7,2168000.0,32.237,NaN,8.13,61.195,2.980608
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,NaN,NaN,NaN,1.7,2168000.0,32.237,NaN,8.13,61.195,2.980608


## Median Age

In [63]:
df_median_age=read_csv_file('1jQ_mcxB7JKIlp5xJ9d4zSmjdJQyAwKp0','median-age',0)
years=df_median_age['Year'].unique()
years=years[1:15]
countries=df_median_age['Code'].unique()
countries = [x for x in countries if str(x) != 'nan']
latest_values=[]

for country in countries:
    max_year=2010
    for year in years:
        if((year > max_year) & (len(df_median_age[(df_median_age['Year']==year) & (df_median_age['Code']==country)]) > 0)):
            max_year = year

    latest_values.append(df_median_age[(df_median_age['Year']==max_year) & (df_median_age['Code']==country)]['UN Population Division (Median Age) (2017)'].values[0])
df_median_age=pd.DataFrame()
df_median_age['Code']=countries
df_median_age['UN Population Division (Median Age) (2017)']=latest_values
df_merge = pd.merge(df_merge, df_median_age, on=["Code"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,doctors_per_1000,nurses_per_1000,beds_per_1000,number_of_arrivals,urban_population,total_vaccinations_per_100,%youth_unemployment_total,life_expectancy,%df_population_gr_65,UN Population Division (Median Age) (2017)
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,NaN,NaN,1.7,2168000.0,32.237,0.518571,8.13,61.195,2.980608,19.600000
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,NaN,NaN,1.7,2168000.0,32.237,0.813333,8.13,61.195,2.980608,19.600000
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,NaN,NaN,1.7,2168000.0,32.237,NaN,8.13,61.195,2.980608,19.600000
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,NaN,NaN,1.7,2168000.0,32.237,NaN,8.13,61.195,2.980608,19.600000


## Excess Mortality  (Previous 25 days)

In [64]:
df_excess_mortality_prev=read_csv_file('1tJBHRAMU_xPLljHaotn4rxeYP5JCkb9j','excess-mortality-p-scores',0)
df_excess_mortality_prev['Date'] = pd.to_datetime(df_excess_mortality_prev['Day']) + pd.to_timedelta(25, unit='d')
df_excess_mortality_prev = df_excess_mortality_prev.groupby(['Code', pd.Grouper(key='Date', freq='W-SUN')]).mean().reset_index()
df_excess_mortality_prev.columns=['Code', 'Date', 'Excess mortality P-scores, all ages Prev 25 days']
df_excess_mortality_prev

,Code,Date,"Excess mortality P-scores, all ages Prev 25 days"
0,ALB,2020-03-01,-10.27
1,ALB,2020-03-29,4.33
2,ALB,2020-04-26,1.52
3,ALB,2020-05-31,5.62
4,ALB,2020-06-28,6.31
...,...,...,...
3029,UZB,2020-09-27,68.99
3030,UZB,2020-10-25,21.22
3031,UZB,2020-11-29,31.14
3032,UZB,2020-12-27,10.71


In [65]:
df_merge = pd.merge(df_merge, df_excess_mortality_prev, on=["Code", "Date"], how="left")
df_merge

## Excess Mortality  (Previous 18 days)

In [67]:
df_excess_mortality_prev=read_csv_file('1tJBHRAMU_xPLljHaotn4rxeYP5JCkb9j','excess-mortality-p-scores',0)
df_excess_mortality_prev['Date'] = pd.to_datetime(df_excess_mortality_prev['Day']) + pd.to_timedelta(18, unit='d')
df_excess_mortality_prev = df_excess_mortality_prev.groupby(['Code', pd.Grouper(key='Date', freq='W-SUN')]).mean().reset_index()
df_excess_mortality_prev.columns=['Code', 'Date', 'Excess mortality P-scores, all ages Prev 18 days']
df_excess_mortality_prev

,Code,Date,"Excess mortality P-scores, all ages Prev 18 days"
0,ALB,2020-02-23,-10.27
1,ALB,2020-03-22,4.33
2,ALB,2020-04-19,1.52
3,ALB,2020-05-24,5.62
4,ALB,2020-06-21,6.31
...,...,...,...
3029,UZB,2020-09-20,68.99
3030,UZB,2020-10-18,21.22
3031,UZB,2020-11-22,31.14
3032,UZB,2020-12-20,10.71


In [68]:
df_merge = pd.merge(df_merge, df_excess_mortality_prev, on=["Code", "Date"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,beds_per_1000,number_of_arrivals,urban_population,total_vaccinations_per_100,%youth_unemployment_total,life_expectancy,%df_population_gr_65,UN Population Division (Median Age) (2017),"Excess mortality P-scores, all ages Prev 25 days","Excess mortality P-scores, all ages Prev 18 days"
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,1.7,2168000.0,32.237,0.518571,8.13,61.195,2.980608,19.600000,NaN,NaN
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,1.7,2168000.0,32.237,0.813333,8.13,61.195,2.980608,19.600000,NaN,NaN
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,1.7,2168000.0,32.237,NaN,8.13,61.195,2.980608,19.600000,NaN,NaN
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,1.7,2168000.0,32.237,NaN,8.13,61.195,2.980608,19.600000,NaN,NaN


## Excess Mortality (Previous 7 days)

In [69]:
df_excess_mortality_prev=read_csv_file('1tJBHRAMU_xPLljHaotn4rxeYP5JCkb9j','excess-mortality-p-scores',0)
df_excess_mortality_prev['Date'] = pd.to_datetime(df_excess_mortality_prev['Day']) + pd.to_timedelta(7, unit='d')
df_excess_mortality_prev = df_excess_mortality_prev.groupby(['Code', pd.Grouper(key='Date', freq='W-SUN')]).mean().reset_index()
df_excess_mortality_prev.columns=['Code', 'Date', 'Excess mortality P-scores, all ages Prev 7 days']
df_excess_mortality_prev

,Code,Date,"Excess mortality P-scores, all ages Prev 7 days"
0,ALB,2020-02-09,-10.27
1,ALB,2020-03-08,4.33
2,ALB,2020-04-12,1.52
3,ALB,2020-05-10,5.62
4,ALB,2020-06-07,6.31
...,...,...,...
3029,UZB,2020-09-13,68.99
3030,UZB,2020-10-11,21.22
3031,UZB,2020-11-08,31.14
3032,UZB,2020-12-13,10.71


In [70]:
df_merge = pd.merge(df_merge, df_excess_mortality_prev, on=["Code", "Date"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,number_of_arrivals,urban_population,total_vaccinations_per_100,%youth_unemployment_total,life_expectancy,%df_population_gr_65,UN Population Division (Median Age) (2017),"Excess mortality P-scores, all ages Prev 25 days","Excess mortality P-scores, all ages Prev 18 days","Excess mortality P-scores, all ages Prev 7 days"
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN,NaN
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN,NaN
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN,NaN
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN,NaN
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,2168000.0,32.237,0.518571,8.13,61.195,2.980608,19.600000,NaN,NaN,NaN
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,2168000.0,32.237,0.813333,8.13,61.195,2.980608,19.600000,NaN,NaN,NaN
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,2168000.0,32.237,NaN,8.13,61.195,2.980608,19.600000,NaN,NaN,NaN
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,2168000.0,32.237,NaN,8.13,61.195,2.980608,19.600000,NaN,NaN,NaN


## Excess Mortality

In [471]:
df_excess_mortality_prev=read_csv_file('1tJBHRAMU_xPLljHaotn4rxeYP5JCkb9j','excess-mortality-p-scores',0)
df_excess_mortality['Date'] = pd.to_datetime(df_excess_mortality['Day'])
df_excess_mortality = df_excess_mortality.groupby(['Code', pd.Grouper(key='Date', freq='W-SUN')]).mean().reset_index()
df_excess_mortality

,Code,Date,"Excess mortality P-scores, all ages"
0,ALB,2020-02-02,-10.27
1,ALB,2020-03-01,4.33
2,ALB,2020-04-05,1.52
3,ALB,2020-05-03,5.62
4,ALB,2020-05-31,6.31
...,...,...,...
3029,UZB,2020-09-06,68.99
3030,UZB,2020-10-04,21.22
3031,UZB,2020-11-01,31.14
3032,UZB,2020-12-06,10.71


In [472]:
df_merge = pd.merge(df_merge, df_excess_mortality, on=["Code", "Date"], how="left")
df_merge

,Code,Date,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,contact_tracing,testing_policy,...,urban_population,total_vaccinations_per_100,%youth_unemployment_total,life_expectancy,%df_population_gr_65,UN Population Division (Median Age) (2017),"Excess mortality P-scores, all ages Prev 36 days","Excess mortality P-scores, all ages Prev 18 days","Excess mortality P-scores, all ages Prev 7 days","Excess mortality P-scores, all ages"
0,ABW,2020-03-08,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,0.000000,...,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN,NaN,NaN
1,ABW,2020-03-15,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,0.000000,...,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN,NaN,NaN
2,ABW,2020-03-22,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,0.000000,...,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN,NaN,NaN
3,ABW,2020-03-29,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,0.285714,...,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN,NaN,NaN
4,ABW,2020-04-05,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,1.000000,...,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7538,ZWE,2021-03-28,-2.326571,23.836714,6.469429,-2.938571,1.020429,-8.775571,0.0,3.000000,...,32.237,0.225714,8.13,61.195,2.980608,19.600000,NaN,NaN,NaN,NaN
7539,ZWE,2021-04-04,5.796143,31.183714,4.000000,12.347000,11.020429,-2.306000,0.0,3.000000,...,32.237,0.250000,8.13,61.195,2.980608,19.600000,NaN,NaN,NaN,NaN
7540,ZWE,2021-04-11,11.979571,38.245143,1.448857,17.347000,16.612286,1.673429,0.0,3.000000,...,32.237,0.294286,8.13,61.195,2.980608,19.600000,NaN,NaN,NaN,NaN
7541,ZWE,2021-04-18,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,3.000000,...,32.237,0.518571,8.13,61.195,2.980608,19.600000,NaN,NaN,NaN,NaN


## Calculation of Raw Deaths

In [474]:
df_excess_mortality_prev=read_csv_file('1tJBHRAMU_xPLljHaotn4rxeYP5JCkb9j','excess-mortality-raw-death-count.csv',0)

df_raw_deaths=df_raw_deaths[['Code', 'Day', 'average_deaths_2015_2019_all_ages']]
df_raw_deaths['month']=pd.to_datetime(df_raw_deaths['Day']).dt.month
df_raw_deaths=df_raw_deaths[['Code', 'month', 'average_deaths_2015_2019_all_ages']]
df_raw_deaths=df_raw_deaths.groupby(['Code','month']).agg('sum').reset_index()
df_raw_deaths['average_deaths_2015_2019_all_ages']=df_raw_deaths['average_deaths_2015_2019_all_ages']/4

df_merge['month']=pd.to_datetime(df_merge['Date']).dt.month
df_merge = pd.merge(df_merge, df_raw_deaths, on=["Code", "month"])
df_merge['deaths_prev_7']=(df_merge['average_deaths_2015_2019_all_ages']/100)*df_merge['Excess mortality P-scores, all ages Prev 7 days']
df_merge['deaths_prev_18']=(df_merge['average_deaths_2015_2019_all_ages']/100)*df_merge['Excess mortality P-scores, all ages Prev 18 days']
df_merge['deaths_prev_25']=(df_merge['average_deaths_2015_2019_all_ages']/100)*df_merge['Excess mortality P-scores, all ages Prev 25 days']
df_merge['deaths']=((df_merge['average_deaths_2015_2019_all_ages']/100)*df_merge['Excess mortality P-scores, all ages'])

df_merge

,Code,Date,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,contact_tracing,testing_policy,...,"Excess mortality P-scores, all ages Prev 36 days","Excess mortality P-scores, all ages Prev 18 days","Excess mortality P-scores, all ages Prev 7 days","Excess mortality P-scores, all ages",month,average_deaths_2015_2019_all_ages,deaths_prev_7,deaths_prev_18,deaths_prev_36,deaths
0,AUS,2020-03-08,1.050000,1.722333,-0.739000,4.977667,-2.089000,7.927667,2.0,1.0,...,5.12,6.18,4.92,4.67,3,3126.250,153.811500,193.202250,160.064000,145.995875
1,AUS,2020-03-15,0.761714,1.510143,-1.286000,7.697286,-0.819857,10.567714,2.0,1.0,...,3.32,4.53,4.67,4.62,3,3126.250,145.995875,141.619125,103.791500,144.432750
2,AUS,2020-03-22,1.285714,3.367429,-1.081714,9.346857,0.938857,10.877857,2.0,1.0,...,4.42,4.92,4.62,7.28,3,3126.250,144.432750,153.811500,138.180250,227.591000
3,AUS,2020-03-29,0.714429,8.979571,-0.387571,5.448857,-2.857143,8.326571,2.0,1.0,...,6.18,4.67,7.28,12.13,3,3126.250,227.591000,145.995875,193.202250,379.214125
4,AUS,2021-03-07,-14.061000,-1.653143,5.306143,-37.918286,-16.959143,-10.775571,2.0,2.0,...,NaN,NaN,NaN,NaN,3,3126.250,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3728,USA,2021-01-31,-23.979714,-11.387857,11.449143,-39.571429,-22.591714,-28.999857,1.0,3.0,...,41.25,36.74,29.53,24.53,1,74249.625,21925.914263,27279.312225,30627.970313,18213.433013
3729,USA,2021-02-07,-24.673429,-12.857143,10.795714,-40.183714,-19.183714,-28.816286,1.0,3.0,...,41.07,33.91,24.53,20.08,2,57847.700,14190.040810,19616.155070,23758.050390,11615.818160
3730,USA,2021-02-14,-25.836714,-14.142714,10.734429,-41.285714,-21.000143,-28.816143,1.0,3.0,...,44.75,29.53,20.08,NaN,2,57847.700,11615.818160,17082.425810,25886.845750,NaN
3731,USA,2021-02-21,-25.938714,-14.551000,10.714000,-41.714286,-25.775714,-27.938714,1.0,3.0,...,36.74,24.53,NaN,NaN,2,57847.700,NaN,14190.040810,21253.244980,NaN


## Calculation of Accumulated Deaths 

In [475]:
df_population = pd.read_csv('/Users/lucas_botella/Downloads/API_SP.POP.TOTL_DS2_en_csv_v2_2252106/API_SP.POP.TOTL_DS2_en_csv_v2_2252106.csv', skiprows=3)

In [476]:
df_population

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,106314.0,NaN,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996973.0,9169410.0,9351441.0,9543205.0,9744781.0,9956320.0,...,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,38041754.0,NaN,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5454933.0,5531472.0,5608539.0,5679458.0,5735044.0,5770570.0,...,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,31825295.0,NaN,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,2854191.0,NaN,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,77142.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,"Population, total",SP.POP.TOTL,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,...,1807106.0,1818117.0,1812771.0,1788196.0,1777557.0,1791003.0,1797085.0,1794248.0,NaN,NaN
260,"Yemen, Rep.",YEM,"Population, total",SP.POP.TOTL,5315355.0,5393036.0,5473671.0,5556766.0,5641597.0,5727751.0,...,24473178.0,25147109.0,25823485.0,26497889.0,27168210.0,27834821.0,28498687.0,29161922.0,NaN,NaN
261,South Africa,ZAF,"Population, total",SP.POP.TOTL,17099840.0,17524533.0,17965725.0,18423161.0,18896307.0,19384841.0,...,52834005.0,53689236.0,54545991.0,55386367.0,56203654.0,57000451.0,57779622.0,58558270.0,NaN,NaN
262,Zambia,ZMB,"Population, total",SP.POP.TOTL,3070776.0,3164329.0,3260650.0,3360104.0,3463213.0,3570464.0,...,14465121.0,14926504.0,15399753.0,15879361.0,16363507.0,16853688.0,17351822.0,17861030.0,NaN,NaN


In [477]:
#Select latest value between 2010 and 2020 
years=df_population.columns[54:-1]
years

Index(['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020'],
      dtype='object')

In [478]:
#Iterate through all values and collect the values for the maximum year recorded for the selected country
latest_values=[]

for index, row in df_population.iterrows():
    
    aux_year=2010
    
    for year in years:
        
        if((math.isnan(row[year])==False)):
            
            aux_year=year
            
    latest_values.append(row[str(aux_year)])

In [479]:
df_population['latest_value']=latest_values
df_population=df_population[['Country Code', 'latest_value']]
df_population.columns=['Code', 'population']
df_population

,Code,population
0,ABW,106314.0
1,AFG,38041754.0
2,AGO,31825295.0
3,ALB,2854191.0
4,AND,77142.0
...,...,...
259,XKX,1794248.0
260,YEM,29161922.0
261,ZAF,58558270.0
262,ZMB,17861030.0


In [480]:
df_merge = pd.merge(df_merge, df_population, on=["Code"], how="left")
df_merge=df_merge.sort_values(['Code','Date'], ascending=[True, True])

In [489]:
df_merge=df_merge.sort_values(['Code','Date'], ascending=[True, True])

In [490]:
accumulated=[]
country='AUS'
cumsum=0

for index, row in df_merge.iterrows():
    country_last=row['Code']
    
    if(country!=country_last):
        
        cumsum=0
        country=row['Code']
        if(row['deaths_prev_18']  > 0):
            cumsum=cumsum+row['deaths_prev_18'] 
        
    else:
        
        if(row['deaths_prev_18']  > 0):
            cumsum=cumsum+row['deaths_prev_18'] 
    
    accumulated.append(cumsum)

df_merge['accumulated']=accumulated
df_merge['accumulated']=100*df_merge['accumulated']/df_merge['population']

## Calculation of R0

In [491]:
df_merge['deaths_prev_7']=df_merge['deaths_prev_7']
df_merge['deaths_prev_18']=df_merge['deaths_prev_18']
df_merge['deaths_prev_36']=df_merge['deaths_prev_36']
df_merge['deaths']=df_merge['deaths']
#df_merge['R0_prev18']=df_merge['deaths_prev_18']/df_merge['deaths_prev_36']
df_merge['R0']=df_merge['deaths']/df_merge['deaths_prev_7']

In [492]:
df_merge

,Code,Date,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,contact_tracing,testing_policy,...,"Excess mortality P-scores, all ages",month,average_deaths_2015_2019_all_ages,deaths_prev_7,deaths_prev_18,deaths_prev_36,deaths,population,accumulated,R0
0,AUS,2020-03-08,1.050000,1.722333,-0.739000,4.977667,-2.089000,7.927667,2.0,1.0,...,4.67,3,3126.25,153.811500,193.202250,160.064000,145.995875,25364307.0,0.000762,0.949187
1,AUS,2020-03-15,0.761714,1.510143,-1.286000,7.697286,-0.819857,10.567714,2.0,1.0,...,4.62,3,3126.25,145.995875,141.619125,103.791500,144.432750,25364307.0,0.001320,0.989293
2,AUS,2020-03-22,1.285714,3.367429,-1.081714,9.346857,0.938857,10.877857,2.0,1.0,...,7.28,3,3126.25,144.432750,153.811500,138.180250,227.591000,25364307.0,0.001926,1.575758
3,AUS,2020-03-29,0.714429,8.979571,-0.387571,5.448857,-2.857143,8.326571,2.0,1.0,...,12.13,3,3126.25,227.591000,145.995875,193.202250,379.214125,25364307.0,0.002502,1.666209
8,AUS,2020-04-05,-0.898000,15.163286,0.448857,-1.061286,-4.959143,6.959000,2.0,1.0,...,11.31,4,2564.75,311.104175,118.491450,116.183175,290.073225,25364307.0,0.002969,0.932399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3680,USA,2021-03-21,-17.347000,-9.142857,8.612286,-33.979571,-10.734714,-24.877286,1.0,3.0,...,NaN,3,70290.60,NaN,NaN,14114.352480,NaN,328239523.0,0.225998,NaN
3681,USA,2021-03-28,-14.673571,-6.510143,7.653000,-32.285571,-3.734571,-24.775286,1.0,3.0,...,NaN,3,70290.60,NaN,NaN,NaN,NaN,328239523.0,0.225998,NaN
3686,USA,2021-04-04,-12.938857,-5.448857,7.449143,-30.163286,3.918429,-25.469429,1.0,3.0,...,NaN,4,53949.95,NaN,NaN,NaN,NaN,328239523.0,0.225998,NaN
3687,USA,2021-04-11,-8.816429,-4.244714,7.040714,-27.183571,4.265286,-25.775714,1.0,3.0,...,NaN,4,53949.95,NaN,NaN,NaN,NaN,328239523.0,0.225998,NaN


In [493]:
df_final=df_merge.dropna(subset=['Excess mortality P-scores, all ages Prev 18 days','Excess mortality P-scores, all ages'])

In [494]:
df_final

,Code,Date,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,contact_tracing,testing_policy,...,"Excess mortality P-scores, all ages",month,average_deaths_2015_2019_all_ages,deaths_prev_7,deaths_prev_18,deaths_prev_36,deaths,population,accumulated,R0
0,AUS,2020-03-08,1.050000,1.722333,-0.739000,4.977667,-2.089000,7.927667,2.0,1.0,...,4.67,3,3126.250,153.811500,193.202250,160.064000,145.995875,25364307.0,0.000762,0.949187
1,AUS,2020-03-15,0.761714,1.510143,-1.286000,7.697286,-0.819857,10.567714,2.0,1.0,...,4.62,3,3126.250,145.995875,141.619125,103.791500,144.432750,25364307.0,0.001320,0.989293
2,AUS,2020-03-22,1.285714,3.367429,-1.081714,9.346857,0.938857,10.877857,2.0,1.0,...,7.28,3,3126.250,144.432750,153.811500,138.180250,227.591000,25364307.0,0.001926,1.575758
3,AUS,2020-03-29,0.714429,8.979571,-0.387571,5.448857,-2.857143,8.326571,2.0,1.0,...,12.13,3,3126.250,227.591000,145.995875,193.202250,379.214125,25364307.0,0.002502,1.666209
8,AUS,2020-04-05,-0.898000,15.163286,0.448857,-1.061286,-4.959143,6.959000,2.0,1.0,...,11.31,4,2564.750,311.104175,118.491450,116.183175,290.073225,25364307.0,0.002969,0.932399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3725,USA,2021-01-10,-17.693857,-4.061286,10.183857,-36.040857,-19.755143,-28.101857,1.0,3.0,...,36.74,1,74249.625,33226.707188,30627.970313,23826.704663,27279.312225,328239523.0,0.179232,0.821006
3726,USA,2021-01-17,-22.775429,-3.979571,12.653143,-38.877714,-16.653143,-42.081714,1.0,3.0,...,33.91,1,74249.625,27279.312225,30494.320988,26752.139888,25178.047837,328239523.0,0.188522,0.922972
3727,USA,2021-01-24,-23.632714,-9.836714,14.061286,-41.020286,-15.857143,-41.714143,1.0,3.0,...,29.53,1,74249.625,25178.047837,33226.707188,30204.747450,21925.914263,328239523.0,0.198645,0.870835
3728,USA,2021-01-31,-23.979714,-11.387857,11.449143,-39.571429,-22.591714,-28.999857,1.0,3.0,...,24.53,1,74249.625,21925.914263,27279.312225,30627.970313,18213.433013,328239523.0,0.206956,0.830681


In [495]:
df_final.to_csv('/Users/lucas_botella/Downloads/df_covid_processed_FINAL.csv')

## Correlation Matrix

In [496]:
import numpy as np

In [497]:
rs = np.random.RandomState(0)
df = pd.DataFrame(rs.rand(10, 10))
corr = df_final.corr()
corr.style.background_gradient(cmap='coolwarm')

## Countries includeed in the analysis

In [403]:
import pycountry
from matplotlib import pyplot as plt

In [404]:
country_list=df_final.Code.unique()
country_list.size

40

In [405]:
def country_name(country):
    return pycountry.countries.get(alpha_3=country).name

In [406]:
def country_code_2_lower(country):
    return pycountry.countries.get(alpha_3=country).alpha_2.lower()

In [407]:
country_name('AUT')

'Austria'

In [408]:
country_code_2_lower('AUT')

'at'

In [409]:
country_names_list=[country_name(country) for country in country_list]
country_names_list

['Australia',
 'Austria',
 'Belgium',
 'Bulgaria',
 'Canada',
 'Switzerland',
 'Chile',
 'Colombia',
 'Czechia',
 'Germany',
 'Denmark',
 'Ecuador',
 'Spain',
 'Estonia',
 'Finland',
 'France',
 'United Kingdom',
 'Greece',
 'Guatemala',
 'Croatia',
 'Hungary',
 'Israel',
 'Italy',
 'Korea, Republic of',
 'Lithuania',
 'Luxembourg',
 'Latvia',
 'Mexico',
 'Malta',
 'Netherlands',
 'Norway',
 'New Zealand',
 'Poland',
 'Portugal',
 'Romania',
 'Slovakia',
 'Slovenia',
 'Sweden',
 'Taiwan, Province of China',
 'United States']

In [410]:
country_names_list_code2_lower=[country_code_2_lower(country) for country in country_list]
country_names_list_code2_lower

['au',
 'at',
 'be',
 'bg',
 'ca',
 'ch',
 'cl',
 'co',
 'cz',
 'de',
 'dk',
 'ec',
 'es',
 'ee',
 'fi',
 'fr',
 'gb',
 'gr',
 'gt',
 'hr',
 'hu',
 'il',
 'it',
 'kr',
 'lt',
 'lu',
 'lv',
 'mx',
 'mt',
 'nl',
 'no',
 'nz',
 'pl',
 'pt',
 'ro',
 'sk',
 'si',
 'se',
 'tw',
 'us']

In [411]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [412]:
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
   
# declare an empty list to store
# latitude and longitude of values 
# of city column
longitude = []
latitude = []
   
# function to find the coordinate
# of a given city 
def findGeocode(city):
       
    # try and catch is used to overcome
    # the exception thrown by geolocator
    # using geocodertimedout  
    try:
          
        # Specify the user_agent as your
        # app name it should not be none
        geolocator = Nominatim(user_agent="your_app_name")
          
        return geolocator.geocode(city)
      
    except GeocoderTimedOut:
          
        return findGeocode(city)    

In [413]:
findGeocode('Spain')

Location(España, (39.3260685, -4.8379791, 0.0))

In [414]:
df_plot_countries= pd.DataFrame()
df_plot_countries['Country']=country_names_list
df_plot_countries.head()

,Country
0,Australia
1,Austria
2,Belgium
3,Bulgaria
4,Canada


In [416]:
# each value from city column
# will be fetched and sent to
# function find_geocode  
latitude=[]
longitude=[]
for i in (df_plot_countries["Country"]):
      
    if findGeocode(i) != None:
           
        loc = findGeocode(i)
          
        # coordinates returned from 
        # function is stored into
        # two separate list
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
       
    # if coordinate for a city not
    # found, insert "NaN" indicating 
    # missing value 
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Taiwan%2C+Province+of+China&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [ ]:
df_plot_countries['latitude']=latitude
df_plot_countries['longitude']=longitude
df_plot_countries

In [ ]:
pip install folium

In [ ]:
# Create a world map to show distributions of users 
import folium
from folium.plugins import MarkerCluster
#empty map
world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)
#for each coordinate, create circlemarker of user percent
for i in range(len(df_plot_countries)):
        lat = df_plot_countries.iloc[i]['latitude']
        long = df_plot_countries.iloc[i]['longitude']
        radius=5
        popup_text = """Country : {}<br>"""
        popup_text = popup_text.format(df_plot_countries.iloc[i]['Country']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster)
#show the map
world_map

In [ ]:
pip install pygal_maps_world

In [ ]:
!pip3 install cairosvg

In [ ]:
import pygal
from pygal.maps.world import World
worldmap_chart = World()

worldmap_chart.title = 'Countries included in the analysis'

for i in range(len(country_names_list)):
    worldmap_chart.add(country_names_list[i], country_names_list_code2_lower[i])
    
worldmap_chart.render_to_file('mymap.html')

In [ ]:
from IPython.core.display import display, HTML

In [ ]:
display(HTML(worldmap_chart.render(is_unicode=True)))